In [5]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/home/angel/Temp/Protein/cafa-5-protein-function-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/angel/Temp/Protein/cafa-5-protein-function-prediction/sample_submission.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/IA.txt
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/go-basic.obo
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/train_terms.tsv
/home/angel/Temp/Protein/cafa-5-protein-function-prediction/Train/train_sequences.fasta
